In [1]:
%load_ext autoreload
%autoreload 2

import logging; logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root))

from src.scraping.guitar_auctions_scraper import GuitarAuctionScraper


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [2]:
scraper = GuitarAuctionScraper(base_url="https://www.guitar-auctions.co.uk")

### Get links for each auction day

In [4]:
auctions_page = scraper.fetch_page("auctions")
# soup = scraper.parse_html(html_content_auctions)

# Define check function for identifying auction links
check_func = lambda href: href.startswith("/sale/")

auction_links = scraper.extract_links(auctions_page, check_func)
for link in auction_links:
    print(link)

2025-11-08 11:41:35,565 - INFO - Using cached page from https://www.guitar-auctions.co.uk/auctions


https://www.guitar-auctions.co.uk/sale/253/the-guitar-auction-%28december%29---day-one---artist-associated-%5Bpreview%5D
https://www.guitar-auctions.co.uk/sale/249/the-guitar-auction-%28december%29---day-one-%5Bpreview%5D


### Get links for each lot

In [ ]:
import re

auction_link = auction_links[1]
page = 1
all_lot_links = []

# lots_page = scraper.fetch_page(auction_link)  # commented out just to stop rerun
# page_lot_links = scraper.extract_links(lots_page, check_func)

# match /sale/<1 to 9 digits>/<1 to 9 digits>
check_func_lots = lambda href: re.match(r'/sale/[1-9]\d*/[1-9]\d*', href) is not None

while True:
    # Construct the URL: first page without query parameter, subsequent pages with ?page=i
    if page == 1:
        lots_page_link = auction_link
    else:
        lots_page_link = f"{auction_link}?page={page}"

    lots_page = scraper.fetch_page(lots_page_link)
    
    logging.info("Fetching links from: %s", lots_page_link)
    # NOTE: check_func here is TOO GENERAL! 'https://www.guitar-auctions.co.uk/sale/253/
    lots_page_lot_links = scraper.extract_links(lots_page, check_func_lots) 
    if not lots_page_lot_links:
        logging.info(
            "No lot links found on page %i. Assuming this is the last page.",
            page
        )
        break
    logging.info(
        "Found %i lots on page %i: %s",
        len(lots_page_lot_links),
        page,
        lots_page_link

    )
    all_lot_links.extend(lots_page_lot_links)
    page += 1

2025-11-08 12:03:07,392 - INFO - Using cached page from https://www.guitar-auctions.co.uk/sale/249/the-guitar-auction-%28december%29---day-one-%5Bpreview%5D
2025-11-08 12:03:07,393 - INFO - Fetching links from: https://www.guitar-auctions.co.uk/sale/249/the-guitar-auction-%28december%29---day-one-%5Bpreview%5D
2025-11-08 12:03:07,402 - INFO - Found 40 lots on https://www.guitar-auctions.co.uk/sale/249/the-guitar-auction-%28december%29---day-one-%5Bpreview%5D
2025-11-08 12:03:07,407 - INFO - Using cached page from https://www.guitar-auctions.co.uk/sale/249/the-guitar-auction-%28december%29---day-one-%5Bpreview%5D?page=2
2025-11-08 12:03:07,411 - INFO - Fetching links from: https://www.guitar-auctions.co.uk/sale/249/the-guitar-auction-%28december%29---day-one-%5Bpreview%5D?page=2
2025-11-08 12:03:07,419 - INFO - Found 40 lots on https://www.guitar-auctions.co.uk/sale/249/the-guitar-auction-%28december%29---day-one-%5Bpreview%5D?page=2
2025-11-08 12:03:07,420 - INFO - Using cached page fr

In [10]:
all_lot_links


['https://www.guitar-auctions.co.uk/sale/253/601/Jeff-Beck--owned-played-and-autographed-1991-Fender-Jeff-Beck-Signature-Stratocaster-electric-guitar-made-in-USA',
 'https://www.guitar-auctions.co.uk/sale/253/602/John-Entwistle-The-Who--John-Deacon-Queen--Owned-and-played-1986-Warwick-Buzzard-bass-guitar-prototype-no5-made-in-West-Germany-ser-no-B00500-86-Body-natural-Zebrano-wood-a-few-small-hairline-splits-minor-play-marks-Neck-wenge-thruneck',
 'https://www.guitar-auctions.co.uk/sale/253/603/Rick-Parfitt-Status-Quo--1975-Travis-Bean-TB1000-Artist-electric-guitar-made-in-USA-ser-no-517',
 'https://www.guitar-auctions.co.uk/sale/253/604/Noel-Gallagher-Oasis-interest--1963-Fender-Stratocaster-electric-guitar-made-in-USA-ser-no-L02403',
 'https://www.guitar-auctions.co.uk/sale/253/606/Damon-Minchella-Ocean-Colour-Scene--Marco-Nelson-Paul-Weller-Young-Disciples-Primal-Scream--Paul-McCartney-Autographed-1970s-Hofner-5001-violin-bass-guitar-made-in-Germany-within-original-Selmer-hard-case-

In [ ]:
page = 1
all_lot_urls = []

while True:
    # Construct the URL: first page without query parameter, subsequent pages with ?page=
    if page == 1:
        preview_url = preview_base_url
    else:
        preview_url = f"{preview_base_url}?page={page}"

    print(f"Fetching page {page}: {preview_url}")
    preview_html = fetch_page(preview_url)
    if not preview_html:
        print("Could not fetch the preview page.")
        break


    # Extract links to individual lot detail pages using the revised approach
    lot_urls = get_lot_links(preview_html, base_url)
    if not lot_urls:
        print(f"No lot links found on page {page}. Assuming this is the last page.")
        break

    print(f"Found {len(lot_urls)} lot(s).")
    all_lot_urls.extend(lot_urls)
    page += 1